In [1]:
import pandas as pd
import os

In [2]:
companies = {
    #Thực phẩm
    "Vinamilk": "VNM",
    "CTCP Sữa TH True Milk": "THM",
    "CTCP Bánh kẹo Bibica": "BBC",
    "CTCP Bao bì Dược": "BMP",
    "CTCP Tập đoàn Masan": "MSN",
    "CTCP Masan Consumer": "MCH",
    "CTCP Thực phẩm Cholimex": "CMF",
    "CTCP Chăn nuôi Mitraco": "MLS",
    "CTCP Chăn nuôi Phú Sơn": "PSL",
    "CTCP Chăn nuôi Tiến Nông": "TNC"
}

In [3]:
# Khởi tạo list chứa dữ liệu từng công ty
all_data = []

# Đọc từng file và gắn mã công ty
for full_name, ticker in companies.items():
    file_path = f"pre-processing/{full_name}_data.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Thêm mã công ty
        df["company_id"] = ticker

        # Chuyển cột thời gian nếu có
        if "quarter" in df.columns:
            df["quarter"] = pd.to_datetime(df["quarter"])

        all_data.append(df)
    else:
        print(f"⚠️ File không tồn tại: {file_path}")

# Gộp toàn bộ dữ liệu
df_all = pd.concat(all_data, ignore_index=True)

# Sắp xếp theo thời gian và công ty
df_all = df_all.sort_values(["company_id", "quarter"])

# Tạo biến mục tiêu P(t+1)
df_all["P_t_plus_1"] = df_all.groupby("company_id")["P"].shift(-1)

# Xoá dòng thiếu dữ liệu đầu vào
df_model = df_all.dropna(subset=["P_t_plus_1", "P/E", "EPS", "D/E", "SIZE"])

df_model.set_index(['company_id', 'quarter'], inplace=True)

df_model

C:\Users\hype1\AppData\Local\Temp\ipykernel_19964\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_19964\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_19964\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_19964\3346459644.py:15: UserWarning: Could not infer for

P       EPS       D/E          P/E       SIZE  \
company_id quarter                                                             
BBC        2019-01-01  25.848744  0.076810  0.219419   336.528876  27.756187   
           2019-04-01  27.537610  0.083082  0.220113   331.452770  27.760721   
           2019-07-01  26.831922  0.130553  0.398124   205.525436  27.917361   
           2019-10-01  29.346354  0.331554  0.545175    88.511621  28.069459   
           2020-01-01  34.002714  0.007125  0.301747  4772.599350  27.899692   
...                          ...       ...       ...          ...        ...   
VNM        2023-04-01  11.903759  0.194049  0.390309    61.343988  31.562234   
           2023-07-01  13.807935  0.119249  0.616544   115.790644  31.637762   
           2023-10-01  12.453304  0.111295  0.503847   111.894640  31.595131   
           2024-01-01  13.197541  0.105010  0.440127   125.678625  31.575586   
           2024-07-01  12.283594  0.115003  0.636495   106.810732  31.665948   

                       P_t_plus_1  
company_id quarter                 
BBC        2019-01-01   27.537610  
           2019-04-01   26.831922  
           2019-07-01   29.346354  
           2019-10-01   34.002714  
           2020-01-01   37.752226  
...                           ...  
VNM        2023-04-01   13.807935  
           2023-07-01   12.453304  
           2023-10-01   13.197541  
           2024-01-01   12.684762  
           2024-07-01   11.936508  

[84 rows x 6 columns]

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Giả sử df_model là DataFrame chứa dữ liệu của bạn
# Khởi tạo StandardScaler
scaler = StandardScaler()

# Chuẩn hóa toàn bộ DataFrame
df_model_scaled = df_model.copy()  # Tạo bản sao để không thay đổi dữ liệu gốc
df_model_scaled[:] = scaler.fit_transform(df_model)

# Kiểm tra lại dữ liệu đã chuẩn hóa
df_model_scaled


P       EPS       D/E       P/E      SIZE  \
company_id quarter                                                        
BBC        2019-01-01 -0.224038 -0.335332 -0.554294 -0.267020 -0.675467   
           2019-04-01 -0.223876 -0.283351 -0.553457 -0.267072 -0.673462   
           2019-07-01 -0.223944  0.110098 -0.338473 -0.268347 -0.604188   
           2019-10-01 -0.223704  1.776023 -0.160880 -0.269531 -0.536922   
           2020-01-01 -0.223259 -0.912894 -0.454868 -0.222108 -0.612002   
...                         ...       ...       ...       ...       ...   
VNM        2023-04-01 -0.225369  0.636366 -0.347912 -0.269806  1.007760   
           2023-07-01 -0.225187  0.016411 -0.074689 -0.269255  1.041162   
           2023-10-01 -0.225317 -0.049515 -0.210792 -0.269295  1.022309   
           2024-01-01 -0.225246 -0.101603 -0.287747 -0.269155  1.013665   
           2024-07-01 -0.225333 -0.018778 -0.050594 -0.269346  1.053627   

                       P_t_plus_1  
company_id quarter                 
BBC        2019-01-01   -0.223363  
           2019-04-01   -0.223433  
           2019-07-01   -0.223183  
           2019-10-01   -0.222720  
           2020-01-01   -0.222347  
...                           ...  
VNM        2023-04-01   -0.224728  
           2023-07-01   -0.224863  
           2023-10-01   -0.224789  
           2024-01-01   -0.224840  
           2024-07-01   -0.224915  

[84 rows x 6 columns]

In [5]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Pooled OLS
pooled_model = PooledOLS(y, X)
pooled_results = pooled_model.fit(cov_type="robust")

# In kết quả ước lượng
pooled_results

PooledOLS Estimation Summary                          
================================================================================
Dep. Variable:             P_t_plus_1   R-squared:                        0.6531
Estimator:                  PooledOLS   R-squared (Between):              0.8778
No. Observations:                  84   R-squared (Within):              -32.907
Date:                Fri, May 09 2025   R-squared (Overall):              0.6531
Time:                        08:56:30   Log-likelihood                   -74.722
Cov. Estimator:                Robust                                           
                                        F-statistic:                      37.186
Entities:                           7   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                    F(4,79)
Min Obs:                       1.0000                                           
Max Obs:                       22.000   F-statistic (robust):             8.2297
                                        P-value                           0.0000
Time periods:                      23   Distribution:                    F(4,79)
Avg Obs:                       3.6522                                           
Min Obs:                       1.0000                                           
Max Obs:                       6.0000                                           
                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const       1.074e-16     0.0663  1.621e-15     1.0000     -0.1319      0.1319
P/E            0.8072     0.2777     2.9062     0.0047      0.2543      1.3600
EPS            0.0556     0.0337     1.6515     0.1026     -0.0114      0.1226
D/E           -0.0592     0.0492    -1.2040     0.2322     -0.1571      0.0387
SIZE           0.0810     0.0572     1.4169     0.1605     -0.0328      0.1948
==============================================================================
PanelResults, id: 0x2771129ad80

In [6]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Fixed Effects (FEM)
fixed_effects_model = PanelOLS(y, X, entity_effects=True)  # entity_effects=True để có hiệu ứng cố định theo công ty
fixed_effects_results = fixed_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
fixed_effects_results


PanelOLS Estimation Summary                           
================================================================================
Dep. Variable:             P_t_plus_1   R-squared:                        0.0404
Estimator:                   PanelOLS   R-squared (Between):             -0.1215
No. Observations:                  84   R-squared (Within):               0.0404
Date:                Fri, May 09 2025   R-squared (Overall):             -0.0443
Time:                        08:56:30   Log-likelihood                    91.921
Cov. Estimator:                Robust                                           
                                        F-statistic:                      0.7683
Entities:                           7   P-value                           0.5493
Avg Obs:                       12.000   Distribution:                    F(4,73)
Min Obs:                       1.0000                                           
Max Obs:                       22.000   F-statistic (robust):             0.6555
                                        P-value                           0.6249
Time periods:                      23   Distribution:                    F(4,73)
Avg Obs:                       3.6522                                           
Min Obs:                       1.0000                                           
Max Obs:                       6.0000                                           
                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const      -4.254e-17     0.0095 -4.487e-15     1.0000     -0.0189      0.0189
P/E           -0.0291     0.0320    -0.9091     0.3663     -0.0927      0.0346
EPS            0.0010     0.0011     0.8483     0.3991     -0.0013      0.0033
D/E            0.0016     0.0025     0.6552     0.5144     -0.0034      0.0066
SIZE          -0.0366     0.0264    -1.3848     0.1703     -0.0893      0.0161
==============================================================================

F-test for Poolability: 630.99
P-value: 0.0000
Distribution: F(6,73)

Included effects: Entity
PanelEffectsResults, id: 0x2773ead3e00

In [7]:
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Random Effects
random_effects_model = RandomEffects(y, X)
random_effects_results = random_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
random_effects_results


RandomEffects Estimation Summary                        
================================================================================
Dep. Variable:             P_t_plus_1   R-squared:                        0.1376
Estimator:              RandomEffects   R-squared (Between):              0.2099
No. Observations:                  84   R-squared (Within):              -1.1222
Date:                Fri, May 09 2025   R-squared (Overall):              0.0291
Time:                        08:56:31   Log-likelihood                   -6.4223
Cov. Estimator:                Robust                                           
                                        F-statistic:                      3.1506
Entities:                           7   P-value                           0.0186
Avg Obs:                       12.000   Distribution:                    F(4,79)
Min Obs:                       1.0000                                           
Max Obs:                       22.000   F-statistic (robust):             0.8200
                                        P-value                           0.5162
Time periods:                      23   Distribution:                    F(4,79)
Avg Obs:                       3.6522                                           
Min Obs:                       1.0000                                           
Max Obs:                       6.0000                                           
                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.3963     0.2920     1.3571     0.1786     -0.1849      0.9775
P/E            0.1239     0.1228     1.0096     0.3158     -0.1204      0.3683
EPS            0.0105     0.0143     0.7311     0.4669     -0.0180      0.0390
D/E           -0.0364     0.0420    -0.8669     0.3886     -0.1200      0.0472
SIZE          -0.1030     0.1518    -0.6787     0.4993     -0.4051      0.1991
==============================================================================
RandomEffectsResults, id: 0x277112ffa10

In [8]:
fixed_effects_results.f_statistic

Model F-statistic (homoskedastic)
H0: All parameters ex. constant are zero
Statistic: 0.7683
P-value: 0.5493
Distributed: F(4,73)
WaldTestStatistic, id: 0x2770f9f7dd0

In [9]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.stats.stattools import durbin_watson
import scipy.linalg as la

# Lấy các hệ số ước lượng và phương sai-covariance của các mô hình
beta_FE = fixed_effects_results.params
beta_RE = random_effects_results.params
var_FE = fixed_effects_results.cov
var_RE = random_effects_results.cov

# Tính toán kiểm định Hausman
diff = beta_FE - beta_RE
diff_cov = var_FE - var_RE

# Kiểm tra tính khả thi của ma trận khác biệt và tính toán giá trị kiểm định Hausman
hausman_stat = diff.T @ la.inv(diff_cov) @ diff
df = len(beta_FE)  # số bậc tự do (số biến độc lập)

# P-value sử dụng phân phối Chi-squared
from scipy.stats import chi2
p_value = 1 - chi2.cdf(hausman_stat, df)

print(f"Hausman Statistic: {hausman_stat}")
print(f"P-value: {p_value}")

Hausman Statistic: -3.3660664737689636
P-value: 1.0


In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Thêm hằng số vào các biến độc lập
X = add_constant(df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']])

# Tính VIF cho từng biến độc lập
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# In kết quả
print(vif_data)

  Variable       VIF
0    const  1.000000
1      P/E  1.039328
2      EPS  1.150179
3      D/E  2.229782
4     SIZE  2.287353


In [11]:
from statsmodels.stats.stattools import durbin_watson

# Kiểm tra tự tương quan cho mô hình Pooled OLS
pooled_resids = pooled_results.resids
print(f'Durbin-Watson statistic for Pooled OLS: {durbin_watson(pooled_resids)}')

# Kiểm tra tự tương quan cho mô hình FEM
fixed_effects_resids = fixed_effects_results.resids
print(f'Durbin-Watson statistic for FEM: {durbin_watson(fixed_effects_resids)}')

# Kiểm tra tự tương quan cho mô hình REM
random_effects_resids = random_effects_results.resids
print(f'Durbin-Watson statistic for REM: {durbin_watson(random_effects_resids)}')


Durbin-Watson statistic for Pooled OLS: 1.4069244173103643
Durbin-Watson statistic for FEM: 2.54576308416939
Durbin-Watson statistic for REM: 0.6172254728309499


In [12]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_bp_test = het_breuschpagan(pooled_resids, X)

print("\n📊 Breusch-Pagan test for Pooled OLS:")
for label, value in zip(labels, pooled_bp_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_bp_test = het_breuschpagan(fixed_effects_resids, X)

print("\n📊 Breusch-Pagan test for FEM:")
for label, value in zip(labels, fixed_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_bp_test = het_breuschpagan(random_effects_resids, X)

print("\n📊 Breusch-Pagan test for REM:")
for label, value in zip(labels, random_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")


📊 Breusch-Pagan test for Pooled OLS:
LM Statistic             : 43.4970
LM P-value               : 0.0000
F-statistic              : 21.2099
F-statistic P-value      : 0.0000

📊 Breusch-Pagan test for FEM:
LM Statistic             : 37.6421
LM P-value               : 0.0000
F-statistic              : 16.0368
F-statistic P-value      : 0.0000

📊 Breusch-Pagan test for REM:
LM Statistic             : 38.2538
LM P-value               : 0.0000
F-statistic              : 16.5153
F-statistic P-value      : 0.0000


In [13]:
from statsmodels.stats.diagnostic import het_white

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_white_test = het_white(pooled_resids, X)

print("\n📊 White test for Pooled OLS:")
for label, value in zip(labels, pooled_white_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_white_test = het_white(fixed_effects_resids, X)

print("\n📊 White test for FEM:")
for label, value in zip(labels, fixed_effects_white_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_white_test = het_white(random_effects_resids, X)

print("\n📊 White test for REM:")
for label, value in zip(labels, random_effects_white_test):
    print(f"{label:<25}: {value:.4f}")



📊 White test for Pooled OLS:
LM Statistic             : 83.8141
LM P-value               : 0.0000
F-statistic              : 2221.8856
F-statistic P-value      : 0.0000

📊 White test for FEM:
LM Statistic             : 75.3066
LM P-value               : 0.0000
F-statistic              : 42.6938
F-statistic P-value      : 0.0000

📊 White test for REM:
LM Statistic             : 83.0526
LM P-value               : 0.0000
F-statistic              : 432.0450
F-statistic P-value      : 0.0000
